In [9]:
from source import seasonalregressor
from source import data_wrangle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import pickle
from sklearn.metrics import mean_squared_error
%matplotlib inline

advanced = data_wrangle.read_all_advanced('./data/advanced')
pergame  = data_wrangle.read_all_pergame('./data/per_game')
seasonal = data_wrangle.clean_and_join_seasonal_dataframe(advanced, pergame)
demographic = data_wrangle.read_demographic_data('./data/player_data.csv')
fullstats = data_wrangle.add_years_in_league(seasonal, demographic)

In [3]:
cols_to_train_from_correlation = ['WS',
 'OWS',
 'VORP',
 'DWS',
 'MP_total',
 'PS/G',
 'FG',
 'MP_pergame',
 'GS',
 'FT',
 '2P',
 'FTA',
 'FGA',
 '2PA',
 'DRB']

In [4]:
with open('train_test.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    trainnames, testnames = pickle.load(f)

In [5]:
sr2 = seasonalregressor.SeasonalRegressor(columns_to_train=cols_to_train_from_correlation)

In [6]:
fulltrain, fulltest = sr2.create_train_test_split(fullstats, trainnames, testnames)

In [7]:
sr2 = sr2.fit(fulltrain)

Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 5}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'PS/G', 'FG', 'MP_pergame', 'GS', 'FT', '2P', 'FTA', 'FGA', '2PA', 'DRB']
Fitting for year: 5
Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 6}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'PS/G', 'FG', 'MP_pergame', 'GS', 'FT', '2P', 'FTA', 'FGA', '2PA', 'DRB']
Fitting for year: 6
Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 7}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'PS/G', 'FG', 'MP_pergame', 'GS', 'FT', '2P', 'FTA', 'FGA', '2PA', 'DRB']
Fitting for year: 7
Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 8}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'P

In [8]:
preds = sr2.predict(fulltest)

Predicting year: 5 for player:  Raef LaFrentz
Predicting year: 5 for player:  Brook Lopez
Predicting year: 5 for player:  Rasual Butler
Predicting year: 5 for player:  Jason Williams
Predicting year: 5 for player:  Jared Jeffries
Predicting year: 5 for player:  Bruce Bowen
Predicting year: 5 for player:  Andrea Bargnani
Predicting year: 5 for player:  Antawn Jamison
Predicting year: 5 for player:  Kirk Hinrich
Predicting year: 5 for player:  Robin Lopez
Predicting year: 5 for player:  Mike Dunleavy
Predicting year: 5 for player:  Brian Cook
Predicting year: 5 for player:  Cuttino Mobley
Predicting year: 5 for player:  Andris Biedrins
Predicting year: 5 for player:  Tyronn Lue
Predicting year: 5 for player:  Goran Dragic
Predicting year: 5 for player:  Shawn Marion
Predicting year: 5 for player:  Vitaly Potapenko
Predicting year: 5 for player:  Steve Nash
Predicting year: 5 for player:  Jake Voskuhl
Predicting year: 5 for player:  Stephon Marbury
Predicting year: 5 for player:  Spencer 

In [10]:
actuals = data_wrangle.get_actuals_for_years_5_thru_9(fullstats, testnames)

In [12]:
predictions = sr2.unpack_prediction_dictionary(preds)


In [13]:
score = mean_squared_error(actuals, predictions)

In [14]:
score

5.950312935483871

In [16]:
fullstats.groupby('Player').mean().head()

,Age,G,MP_total,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Seasons_number
Player,,,,,,,,,,,,,,,,,,,,,
A.C. Green,35.0,75.800000,1881.000000,12.020000,0.4868,0.021600,0.300200,9.420000,18.560000,13.980000,...,2.060000,4.020000,6.060000,0.860000,0.680000,0.200000,0.640000,1.600000,5.78,14.0
A.J. Bramlett,23.0,8.000000,61.000000,-0.400000,0.1900,0.000000,0.000000,21.700000,18.500000,20.100000,...,1.500000,1.300000,2.800000,0.000000,0.100000,0.000000,0.400000,1.600000,1.00,1.0
A.J. Guyton,23.0,26.666667,415.333333,4.366667,0.3240,0.371000,0.068333,1.366667,3.800000,2.533333,...,0.200000,0.500000,0.700000,1.566667,0.333333,0.133333,0.666667,0.533333,3.80,2.0
A.J. Hammons,24.0,22.000000,163.000000,8.400000,0.4720,0.238000,0.476000,5.400000,21.000000,12.800000,...,0.400000,1.300000,1.600000,0.200000,0.000000,0.600000,0.500000,1.000000,2.20,1.0
A.J. Price,25.5,43.500000,654.833333,11.716667,0.4765,0.473667,0.172333,1.883333,9.033333,5.433333,...,0.233333,1.083333,1.333333,2.000000,0.433333,0.033333,0.800000,0.816667,5.35,3.5


In [18]:
np.average(fullstats["PER"], weights=fullstats["Seasons_number"])

13.019147856103423

In [21]:
from source.helper_functions import weighted_mean_one_col_weight as wm

In [22]:
fullstats.groupby('Player').apply(wm, cols_to_train_from_correlation)

,WS,OWS,VORP,DWS,MP_total,PS/G,FG,MP_pergame,GS,FT,2P,FTA,FGA,2PA,DRB
Player,,,,,,,,,,,,,,,
A.C. Green,3.871429,1.687143,0.354286,2.184286,1839.828571,5.670000,2.338571,23.808571,49.942857,1.037143,2.338571,1.560000,5.142857,5.065714,3.921429
A.J. Bramlett,-0.200000,-0.200000,-0.200000,0.100000,61.000000,1.000000,0.500000,7.600000,0.000000,0.000000,0.500000,0.000000,2.600000,2.600000,1.300000
A.J. Guyton,0.033333,0.016667,-0.233333,-0.016667,311.833333,2.800000,1.066667,9.933333,3.333333,0.250000,0.550000,0.283333,3.766667,2.266667,0.366667
A.J. Hammons,0.000000,-0.200000,-0.100000,0.200000,163.000000,2.200000,0.800000,7.400000,0.000000,0.400000,0.500000,0.900000,1.900000,1.500000,1.300000
A.J. Price,0.690476,0.176190,-0.061905,0.461905,557.619048,4.828571,1.800000,12.761905,4.428571,0.585714,1.152381,0.819048,4.747619,2.566667,0.990476
Aaron Brooks,1.447541,0.759016,-0.073770,0.693443,1171.295082,7.652459,2.803279,17.240984,12.245902,1.052459,1.770492,1.231148,6.824590,4.009836,1.078689
Aaron Gordon,3.450000,1.630000,0.980000,1.820000,1905.300000,13.210000,4.970000,28.250000,52.600000,2.110000,3.770000,3.000000,11.120000,7.310000,4.800000
Aaron Gray,0.757143,0.121429,-0.017857,0.660714,526.000000,2.946429,1.214286,12.146429,14.250000,0.467857,1.214286,0.864286,2.392857,2.392857,2.435714
Aaron Harrison,-0.083333,-0.216667,-0.066667,0.116667,137.666667,3.566667,1.083333,14.816667,1.500000,0.800000,0.566667,1.183333,4.266667,1.650000,1.383333
